In [1]:
# creating groups of 64 slices

in_path_labels = "E:/My Projects For PortFolio/AI CV Projects/medical_image_segmentation/model_training/Software/Dataset/DicomFiles/Labels"
out_path_labels = "E:\My Projects For PortFolio\AI CV Projects\medical_image_segmentation\model_training\Software\Dataset\DicomGroups\labels"

In [2]:
in_path_images = "E:\My Projects For PortFolio\AI CV Projects\medical_image_segmentation\model_training\Software\Dataset\DicomFiles\Images"
out_path_images = "E:\My Projects For PortFolio\AI CV Projects\medical_image_segmentation\model_training\Software\Dataset\DicomGroups\images"

In [3]:
# importing the libraries

from glob import glob
import shutil
import os
import dicom2nifti

# taking slices of the image

In [20]:
for patient in glob(in_path_labels + "/*"):
    patient_name = os.path.basename(os.path.normpath(patient))
    number_folder = int(len(glob(patient + "/*"))/ 64)

    for i in range(number_folder):
        output_path_name = os.path.join(out_path_labels, patient_name + "_" + str(i))

        os.mkdir(output_path_name)
        for i, file in enumerate(glob(patient + "/*")):
            if i == 64 + 1:
                break
            shutil.move(file, output_path_name)

In [4]:
# convert the dicom files into nifties

in_image_path = "E:\My Projects For PortFolio\AI CV Projects\medical_image_segmentation\model_training\Software\Dataset\DicomGroups\images\*"
in_label_path = "E:\My Projects For PortFolio\AI CV Projects\medical_image_segmentation\model_training\Software\Dataset\DicomGroups\labels\*"

In [5]:
list_images = glob(in_image_path)
list_labels = glob(in_label_path)

In [6]:
image_out = "E:/My Projects For PortFolio/AI CV Projects/medical_image_segmentation/model_training/Software/Dataset/Nifti_pre_processed/images"
labels_out = "E:/My Projects For PortFolio/AI CV Projects/medical_image_segmentation/model_training/Software/Dataset/Nifti_pre_processed/labels"

In [30]:
for patient in list_labels:
    patient_name = os.path.basename(os.path.normpath(patient))
    dicom2nifti.dicom_series_to_nifti(patient, os.path.join(labels_out, patient_name + ".nii.gz"))

# Finding the Empty .nii files with no labels

In [34]:
import nibabel as nib
import numpy as np

In [35]:
input_nifti_file_path = "E:/My Projects For PortFolio/AI CV Projects/medical_image_segmentation/model_training/Software/Dataset/Nifti_pre_processed/labels/*"
list_labels = glob(input_nifti_file_path)

for patient in list_labels:
    nifti_file = nib.load(patient)
    f_data = nifti_file.get_fdata()
    np_unique = np.unique(f_data)
    if len(np_unique) ==1:
        print(patient)

E:/My Projects For PortFolio/AI CV Projects/medical_image_segmentation/model_training/Software/Dataset/Nifti_pre_processed/labels\patient_000_label_0.nii.gz
E:/My Projects For PortFolio/AI CV Projects/medical_image_segmentation/model_training/Software/Dataset/Nifti_pre_processed/labels\patient_000_label_1.nii.gz
E:/My Projects For PortFolio/AI CV Projects/medical_image_segmentation/model_training/Software/Dataset/Nifti_pre_processed/labels\patient_000_label_6.nii.gz
E:/My Projects For PortFolio/AI CV Projects/medical_image_segmentation/model_training/Software/Dataset/Nifti_pre_processed/labels\patient_000_label_7.nii.gz
E:/My Projects For PortFolio/AI CV Projects/medical_image_segmentation/model_training/Software/Dataset/Nifti_pre_processed/labels\patient_001_label_0.nii.gz
E:/My Projects For PortFolio/AI CV Projects/medical_image_segmentation/model_training/Software/Dataset/Nifti_pre_processed/labels\patient_001_label_3.nii.gz
E:/My Projects For PortFolio/AI CV Projects/medical_image_

In [6]:
import os
from glob import glob
import shutil
from tqdm import tqdm
import dicom2nifti
import numpy as np
import nibabel as nib
from monai.transforms import (
    LoadImage,
    Resized,
    ToTensor,
    Spacing,
    Orientation,
    ScaleIntensityRange,
    CropForeground,
)

from monai.data import DataLoader, Dataset, CacheDataset
from monai.utils import set_determinism

In [12]:
from monai.transforms import AddChanneld

ImportError: cannot import name 'AddChanneld' from 'monai.transforms' (C:\Users\Admin\anaconda3\envs\med_seg_kidney\lib\site-packages\monai\transforms\__init__.py)

In [8]:
def create_groups(in_dir, out_dir, Number_slices):
    '''
    This function is to get the last part of the path so that we can use it to name the folder.
    `in_dir`: the path to your folders that contain dicom files
    `out_dir`: the path where you want to put the converted nifti files
    `Number_slices`: here you put the number of slices that you need for your project and it will 
    create groups with this number.
    '''

    for patient in glob(in_dir + '/*'):
        patient_name = os.path.basename(os.path.normpath(patient))

        # Here we need to calculate the number of folders which mean into how many groups we will divide the number of slices
        number_folders = int(len(glob(patient + '/*')) / Number_slices)

        for i in range(number_folders):
            output_path = os.path.join(out_dir, patient_name + '_' + str(i))
            os.mkdir(output_path)

            # Move the slices into a specific folder so that you will save memory in your desk
            for i, file in enumerate(glob(patient + '/*')):
                if i == Number_slices + 1:
                    break
                
                shutil.move(file, output_path)

In [9]:
def dcm2nifti(in_dir, out_dir):
    '''
    This function will be used to convert dicoms into nifti files after creating the groups with 
    the number of slices that you want.
    `in_dir`: the path to the folder where you have all the patients (folder of all the groups).
    `out_dir`: the path to the output, which means where you want to save the converted nifties.
    '''

    for folder in tqdm(glob(in_dir + '/*')):
        patient_name = os.path.basename(os.path.normpath(folder))
        dicom2nifti.dicom_series_to_nifti(folder, os.path.join(out_dir, patient_name + '.nii.gz'))

In [10]:
def find_empy(in_dir):
    '''
    This function will help you to find the empty volumes that you may not need for your training
    so instead of opening all the files and search for the empty ones, them use this function to make it quick.
    '''
    
    list_patients = []
    for patient in glob(os.path.join(in_dir, '*')):
        img = nib.load(patient)

        if len(np.unique(img.get_fdata())) > 2:
            print(os.path.basename(os.path.normpath(patient)))
            list_patients.append(os.path.basename(os.path.normpath(patient)))
    
    return list_patients

In [13]:
def prepare(in_dir, pixdim=(1.5, 1.5, 1.0), a_min=-200, a_max=200, spatial_size=[128,128,64], cache=False):

    """
    This function is for preprocessing, it contains only the basic transforms, but you can add more operations that you 
    find in the Monai documentation.
    https://monai.io/docs.html
    """

    set_determinism(seed=0)

    path_train_volumes = sorted(glob(os.path.join(in_dir, "TrainVolumes", "*.nii.gz")))
    path_train_segmentation = sorted(glob(os.path.join(in_dir, "TrainSegmentation", "*.nii.gz")))

    path_test_volumes = sorted(glob(os.path.join(in_dir, "TestVolumes", "*.nii.gz")))
    path_test_segmentation = sorted(glob(os.path.join(in_dir, "TestSegmentation", "*.nii.gz")))

    train_files = [{"vol": image_name, "seg": label_name} for image_name, label_name in zip(path_train_volumes, path_train_segmentation)]
    test_files = [{"vol": image_name, "seg": label_name} for image_name, label_name in zip(path_test_volumes, path_test_segmentation)]

    train_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            AddChanneld(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            ScaleIntensityRanged(keys=["vol"], a_min=a_min, a_max=a_max, b_min=0.0, b_max=1.0, clip=True), 
            CropForegroundd(keys=["vol", "seg"], source_key="vol"),
            Resized(keys=["vol", "seg"], spatial_size=spatial_size),   
            ToTensord(keys=["vol", "seg"]),

        ]
    )

    test_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            AddChanneld(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            ScaleIntensityRanged(keys=["vol"], a_min=a_min, a_max=a_max,b_min=0.0, b_max=1.0, clip=True), 
            CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
            Resized(keys=["vol", "seg"], spatial_size=spatial_size),   
            ToTensord(keys=["vol", "seg"]),

            
        ]
    )

    if cache:
        train_ds = CacheDataset(data=train_files, transform=train_transforms,cache_rate=1.0)
        train_loader = DataLoader(train_ds, batch_size=1)

        test_ds = CacheDataset(data=test_files, transform=test_transforms, cache_rate=1.0)
        test_loader = DataLoader(test_ds, batch_size=1)

        return train_loader, test_loader

    else:
        train_ds = Dataset(data=train_files, transform=train_transforms)
        train_loader = DataLoader(train_ds, batch_size=1)

        test_ds = Dataset(data=test_files, transform=test_transforms)
        test_loader = DataLoader(test_ds, batch_size=1)

        return train_loader, test_loader


In [15]:
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.losses import DiceLoss, DiceCELoss
import torch